# Offspect collimated reduction

In [ ]:
import scipp as sc
import scippneutron as scn
import plopp as pp
from ess import amor, reflectometry
import numpy as np
import ess

pp.patch_scipp()
%matplotlib widget

In [ ]:
logger = ess.logging.configure_workflow('offspec_reduction',
                                        filename=None)

In [ ]:
sample = scn.load("../../../../data/OFFSPEC00062118.nxs", mantid_args={"LoadMonitors": True})

In [ ]:
direct_beam = scn.load("../../../../data/OFFSPEC00062163.nxs", mantid_args={"LoadMonitors": True},)

In [ ]:
sample

TODO use actual metadata

In [ ]:
from orsopy import fileio
from ess.amor.orso import make_orso

owner = fileio.base.Person('Jochen Stahn', 'Paul Scherrer Institut', 'jochen.stahn@psi.ch')
creator = fileio.base.Person('Andrew R. McCluskey', 'European Spallation Source', 'andrew.mccluskey@ess.eu')

orso = make_orso(owner=owner,
                 sample=fileio.data_source.Sample('Ni/Ti Multilayer', 'gas/solid', 'air | (Ni | Ti) * 5 | Si'),
                 creator=creator,
                 reduction_script='https://github.com/scipp/ess/blob/main/docs/instruments/amor/amor_reduction.ipynb')

In [ ]:
sample.attrs['orso'] = sc.scalar(orso)
direct_beam.attrs['orso'] = sc.scalar(orso)

In [ ]:
sample.hist(tof=200).plot()

In [ ]:
sample.hist(tof=200).sum('spectrum').plot()

In [ ]:
direct_beam.hist(tof=200).sum('spectrum').plot()

sample and direct_beam are misaligned in spectrum.

In [ ]:
s = sample.hist(tof=200)
d = direct_beam.hist(tof=s.coords['tof'])
r = s / d
r.plot()

In [ ]:
graph = {**reflectometry.conversions.specular_reflection()}

In [ ]:
wavelength_edges = sc.linspace('wavelength', 0.8, 14.0, 1000, unit='Å')
w_sample = reflectometry.conversions.tof_to_wavelength(sample, wavelength_edges)

In [ ]:
w_sample.hist().plot()

In [ ]:
w_direct_beam = reflectometry.conversions.tof_to_wavelength(direct_beam, wavelength_edges)

Align sample and direct_beam by cropping both in spectrum around the specular peak.
Then pretend that the direct beam was measured at the same pixels ('spectrum' and 'position') as the sample.

In [ ]:
sample_specular_pixel = np.argmax(w_sample.sum('wavelength').values) + w_sample.coords['spectrum'].min().value
direct_beam_specular_pixel = np.argmax(w_direct_beam.sum('wavelength').values) + w_direct_beam.coords['spectrum'].min().value
width = 15
cropped_sample = w_sample['spectrum', sc.index(sample_specular_pixel-width): sc.index(sample_specular_pixel+width)].copy()
cropped_direct_beam = w_direct_beam['spectrum', sc.index(direct_beam_specular_pixel-width): sc.index(direct_beam_specular_pixel+width)].copy()
cropped_direct_beam.coords['spectrum'] = cropped_sample.coords['spectrum'].copy()
cropped_direct_beam.attrs['position'] = cropped_sample.attrs['position'].copy()

TODO normalise sample and direct_beam by current (proton charge) and monitor 2 (monitor 1 and 3 are unused)

In [ ]:
ref = sc.values(cropped_direct_beam).hist()
w_norm = cropped_sample / ref
w_norm.masks['no_reference_neutrons'] = (ref == sc.scalar(0, unit='count')).data
w_norm.coords['sample_rotation'] = sample.attrs['Theta'].value[-1].data

In [ ]:
w_norm.hist().plot()

In [ ]:
w_norm

Gravity seems to be in -y direction.

In [ ]:
x = w_norm.copy(deep=False)
x.attrs['gravity'] = sc.vector([0, -1, 0]) * sc.constants.g
norm_q = reflectometry.conversions.theta_to_q(x, q_edges=sc.geomspace('Q', 0.0004, 0.016, 200, unit='1/Å'), graph=graph)

In [ ]:
norm_q.hist().plot(norm='log')

In [ ]:
norm_q.hist().sum('spectrum').plot(norm='log')